# Interacting particles in a box

In [1]:
import CairoMakie as cm

include("../src/verlet-new.jl")

calc_lj_hamiltonian (generic function with 1 method)

In [2]:
dt = 0.000_1
cutoff = 2.5
box_size = 30.0
pos_arr, v_arr = verlet_simulate(N_particles=100, steps=50_000,L=box_size, dt=dt,cutoff=cutoff)

UndefVarError: UndefVarError: `norm` not defined

In [3]:
v = sb.mean(v_arr,dims=2)
v = reshape(v,length(v))
t = collect(range(0,length(v)*dt,length(v)));

UndefVarError: UndefVarError: `v_arr` not defined

In [4]:
fig = cm.Figure()
ax = cm.Axis(fig[1, 1], xlabel="Time", ylabel="Velocity")
cm.plot!(ax, t, v)

fig

UndefVarError: UndefVarError: `t` not defined

In [5]:
energy = calc_lj_hamiltonian(pos_arr, v_arr, dt=dt, cutoff=cutoff,L=box_size)
t = collect(range(0,length(energy)*dt,length(energy)));

In [ ]:
fig = cm.Figure()
ax = cm.Axis(fig[1,1],xlabel="time", ylabel="energy")

cm.plot!(ax, t, energy)


# cm.xlims!(ax, 0, 0.2)
# cm.ylims!(ax, 8000, 8300)


fig